In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)


## Alignment matrices for final production

Cancer matrices

In [2]:
def create_clean_matrices(onco_in, ts_in, mut_out, aff_out):
    patient_mutations = pd.concat([pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/oncogenes.patient_mutation.csv', index_col=0).transpose(),
                                pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/tsgenes.patient_mutation.csv', index_col=0).transpose(),
                                pd.read_csv('/cellar/users/ramarty/Data/hla/mutations/matrices/indels.patient_mutation.csv', index_col=0).transpose()]).transpose()
    patient_affinites = pd.concat([pd.read_csv(onco_in, index_col=0),
                                pd.read_csv(ts_in, index_col=0)]).transpose()
    print patient_affinites.shape, patient_mutations.shape
    # restrict to overlapping patients
    patients = [x for x in list(patient_affinites.index) if x in list(patient_mutations.index)]
    mutations = [x for x in list(patient_affinites.columns) if x in list(patient_mutations.columns)]
    patient_affinites = patient_affinites.ix[patients, mutations]
    patient_mutations = patient_mutations.ix[patients, mutations]
    print patient_affinites.shape, patient_mutations.shape
    # restrict to mutations >= 3 occurrences
    mutations = list(patient_mutations.sum()[patient_mutations.sum() > 2].index)
    print len(mutations)
    patient_affinites = patient_affinites.ix[:, mutations]
    patient_mutations = patient_mutations.ix[:, mutations]
    print patient_affinites.shape, patient_mutations.shape
    patient_affinites.to_csv(mut_out)
    patient_mutations.to_csv(aff_out)

In [4]:
for gene in ['DR', 'DP', 'DQ', 'all']:
    print gene
    create_clean_matrices('/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/oncogenes.{0}.csv'.format(gene),
                          '/cellar/users/ramarty/Data/hla_ii/presentation/patient_matrices/tsgenes.{0}.csv'.format(gene),
                          '/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.cancer.{0}.csv'.format(gene),
                          '/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_mutations.cancer.{0}.csv'.format(gene))

DR
(7842, 1245) (10177, 1585)
(7656, 1243) (7656, 1243)
762
(7656, 762) (7656, 762)
DP
(7745, 1245) (10177, 1585)
(7579, 1243) (7579, 1243)
700
(7579, 700) (7579, 700)
DQ
(8097, 1245) (10177, 1585)
(7916, 1243) (7916, 1243)
760
(7916, 760) (7916, 760)
all
(7290, 1245) (10177, 1585)
(7137, 1243) (7137, 1243)
662
(7137, 662) (7137, 662)


Germline variants

In [3]:
df = pd.read_csv('/cellar/users/ramarty/Data/hla/git_data/data/patient_affinities.germline.PHBR.csv',index_col=0)

In [8]:
df.head()

,PLEKHN1_R487P,C1orf170_E599Q,C1orf170_Q426E,C1orf170_W20R,HES4_R44S,RNF223_A172V,MIB2_F72L,CDK11B_H112R,CDK11B_R93W,C1orf222_K136E,...,HLA-DMB_T197I,HLA-DPB1_A65V,TAPBP_T260R,ZBTB22_T310A,MLN_V15A,CLPSL2_R79C,PNPLA1_S522P,C6orf222_L582V,CCDC167_I97V,GLP1R_L260F
TCGA-02-0003,2.147505,5.581602,4.409719,0.226405,4.758492,1.768935,3.833495,1.823444,4.359657,3.659164,...,1.041915,0.180261,0.129225,3.334881,3.324347,2.012772,4.765033,1.164469,0.942998,0.322148
TCGA-02-0047,3.365439,1.283630,27.645346,9.642965,18.126939,3.716492,4.589671,13.550899,2.154968,3.852229,...,8.829950,0.564263,1.714286,12.728177,0.439560,5.910448,21.564205,1.191359,0.755489,0.227194
TCGA-02-0055,2.188815,2.566543,3.661706,4.370128,5.865712,1.058662,3.301730,0.563452,1.167109,2.417022,...,4.622874,0.747664,1.731499,1.016050,1.405662,5.917889,1.765260,1.114689,6.452816,0.240964
TCGA-02-2466,3.139742,4.185836,22.070916,3.623957,6.695494,6.590063,2.587108,1.313797,2.534721,6.058137,...,2.495603,0.467638,1.582355,10.456508,1.978573,3.565097,2.434783,1.457187,9.822040,0.401731
TCGA-02-2470,4.405529,6.227380,6.780822,15.219512,30.838296,5.054079,2.846873,4.975901,1.184211,0.932335,...,2.823233,0.407628,1.882611,17.474453,2.921072,3.529412,23.250243,1.701553,0.901734,0.199496


In [4]:
df.shape

(3657, 1000)

In [6]:
# get the allele matrix
allele_df = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/allele_matrices/common.csv', index_col=0)

In [7]:
allele_df.head()

,mutation,DRB1_0101,DRB1_0102,DRB1_0103,DRB1_0104,DRB1_0105,DRB1_0106,DRB1_0107,DRB1_0108,DRB1_0109,...,HLA-DQA10602-DQB10635,HLA-DQA10602-DQB10636,HLA-DQA10602-DQB10637,HLA-DQA10602-DQB10638,HLA-DQA10602-DQB10639,HLA-DQA10602-DQB10640,HLA-DQA10602-DQB10641,HLA-DQA10602-DQB10642,HLA-DQA10602-DQB10643,HLA-DQA10602-DQB10644
0,A1CF_L431S,21.0,18.0,15.0,16.0,21.0,15.0,21.0,21.0,17.0,...,24.0,20.0,16.0,20.0,20.0,16.0,16.0,17.0,24.0,16.0
1,A1CF_Q25R,16.0,7.5,14.0,9.0,16.0,10.0,16.0,16.0,17.0,...,8.5,19.0,9.5,19.0,19.0,11.0,11.0,18.0,8.5,11.0
2,A1CF_R399H,17.0,21.0,22.0,23.0,17.0,20.0,17.0,17.0,11.0,...,5.0,60.0,23.0,60.0,60.0,20.0,20.0,47.0,5.0,20.0
3,A1CF_S154L,60.0,44.0,48.0,46.0,60.0,60.0,60.0,60.0,70.0,...,36.0,30.0,37.0,30.0,30.0,37.0,37.0,32.0,36.0,37.0
4,A2ML1_T90I,3.5,4.0,7.5,7.0,3.5,4.0,3.5,3.5,3.5,...,7.0,22.0,7.5,22.0,22.0,10.0,10.0,14.0,7.0,10.0


Passenger mutations